In [ ]:
import re
import math
import pickle

from sqlalchemy import text
from collections import defaultdict, defaultdict, Counter
from tqdm.notebook import tqdm

from konlpy.tag import Okt
okt = Okt()

from utils.DB_connect import get_engine
engine = get_engine()

In [ ]:
def preprocess_txt(corpus: str):
    # remove tab
    txt_clean = re.sub(r'\s+', ' ', corpus).strip()
    # remove NBSP
    txt_clean = txt_clean.replace('\xa0', ' ')
    # remove invis char
    txt_clean = txt_clean.replace('\u200b', '')
    # remove ZWSP
    txt_clean = re.sub(r'[\u200B]', '', txt_clean)
    # remove html elements
    txt_clean = re.sub(r'<.*?>', '', txt_clean)

    # norm=True applies orthographic normalization
    # stem=True reduces to word stem
    lemmas = [(token, pos) for token, pos in okt.pos(txt_clean, norm=True, stem=True)]
    # 특수문자 제거
    lemmas_clean = [(re.sub(r'[^가-힣a-zA-Z\s]', '', t), p) for t, p in lemmas]
    # 한개짜리 단어 제거
    lemmas_clean = [(t, p) for t, p in lemmas_clean if len(t) > 1]
    
    use_pos = [
        'Noun',
        #'Verb',
        # 'Adjective',
        'Hashtag',
    ]
    lemmas_clean = [kw for kw, pos in lemmas_clean
                    if pos in use_pos]

    word_counts = {w: c for w, c in Counter(lemmas_clean).items()}

    return word_counts

In [3]:
with engine.connect() as conn:
    data_len = conn.execute(text(
        "SELECT COUNT(*) FROM library_reviews"
    )).all()[0][0]
    result = conn.execute(text(
        "SELECT `library_id`, `review` FROM library_reviews"
    ))

review_processed = [([row[0], preprocess_txt(row[1])]) for row in tqdm(result, total=data_len)]

  0%|          | 0/17313 [00:00<?, ?it/s]

In [4]:
review_processed[0]

[5,
 {'용흥': 1,
  '중앙': 1,
  '용흥동': 2,
  '분양': 2,
  '전환': 1,
  '아파트': 1,
  '가이드': 1,
  '포항시': 1,
  '북구': 1,
  '사업': 1,
  '골프': 1,
  '연습장': 1,
  '도서관': 1,
  '사우나': 1,
  '여러': 1,
  '가지': 1,
  '공간': 1,
  '계획': 1,
  '코인': 1,
  '세탁실': 1}]

In [5]:
def get_tf_idf(docs: list[dict]):
    _word_list = list(set(  # 중복제거
        [k for blog in docs for k, _ in blog.items()]
    ))

    df_scores = [(w, len([True for blog in docs if w in blog])) for w in _word_list]

    doc_len = len(docs)
    idf_scores = [(w, math.log(doc_len / (1 + df)) ) for w, df in df_scores]

    total_tf_idf_top = list()
    for blog in docs:  # 각 document(blog)에 대한 TF-IDF점수를 구할 수 있음.
        tf_scores = [(w, blog.get(w, 0)) for w in _word_list]
        tfidf_scores = [(tf[0], tf[1] * idf[1]) for tf, idf in zip(tf_scores, idf_scores) if tf[1] > 0]

        tfidf_scores_top = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)[:10]  # top 10
        total_tf_idf_top += tfidf_scores_top

    total_tf_idf_top.sort(key=lambda x: x[1], reverse=True)
    return total_tf_idf_top[:5]

In [7]:
get_tf_idf([v for _, v in review_processed[-10:]])

[('장애인', 4.828313737302301),
 ('장평', 3.6119184129778086),
 ('시립', 3.6119184129778086),
 ('프로그램', 3.2188758248682006),
 ('지원', 3.2188758248682006)]

In [10]:
grouped = defaultdict(list)
for lib_id, review in review_processed:
    grouped[lib_id].append(review)

In [11]:
tfidf_per_lib = {i: get_tf_idf(lib_review) for i, lib_review in grouped.items()}

In [12]:
[print(tfidf_per_lib[n]) for n in range(1, 25)]
print()

[('버드', 2.7488721956224653), ('개나리', 1.8325814637483102), ('중앙', 1.8325814637483102), ('무척', 0.9162907318741551), ('카페', 0.9162907318741551)]
[('강동', 1.8325814637483102), ('이용', 1.8325814637483102), ('성내', 1.0216512475319814), ('별관', 1.0216512475319814), ('강동구', 1.0216512475319814)]
[('설치', 1.8325814637483102), ('참여', 1.8325814637483102), ('부대', 1.8325814637483102), ('해병대', 1.8325814637483102), ('자원봉사', 1.8325814637483102)]
[('여기', 1.8325814637483102), ('서울시', 1.8325814637483102), ('호공', 1.0216512475319814), ('따릉', 0.9162907318741551), ('강가', 0.9162907318741551)]
[('재개발', 1.8325814637483102), ('대수', 1.8325814637483102), ('세대', 1.8325814637483102), ('설명', 1.8325814637483102), ('주민', 1.5324768712979722)]
[('아주', 1.8325814637483102), ('평생', 0.9162907318741551), ('관일', 0.9162907318741551), ('일요일', 0.9162907318741551), ('동작', 0.9162907318741551)]
[('전쟁', 2.7488721956224653), ('얼굴', 2.7488721956224653), ('도서', 1.8325814637483102), ('진행', 1.8325814637483102), ('수업', 1.8325814637483102)]
[('서비

In [ ]:
with open('./data/tfidf_score.pkl', 'wb') as f:
    pickle.dump(tfidf_per_lib, f)